## Week 3 Assignment

#### Task

In the course material, a simple convolutional neural network is built, trained and tested to solve the multiclass classification task presented by the CIFAR-10 dataset. To improve the accuracy, you should experiment with pre trained models. Follow the instructions in Chollet's book "Deep Learning with Python", 2nd edition, Chapter 8, pp. 225-231: Feature extraction with a pre trained model. Pick one of the pre trained models available with Keras, and discard the Dense classifier top. You should only use the convolution base to preprocess the original images to a new representation, using its predict method. For this modified input data, you should build a simple fully connected classifier, train it, and test it.



In [1]:
from keras.datasets import cifar10
import numpy as np
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=77)



#### Model Creation and Image processing

In [ ]:
from keras.layers import Resizing
from keras.applications.vgg16 import preprocess_input
from keras.applications import VGG16


# Define feature extractor
base_model = VGG16(include_top=False, weights="imagenet")
base_model.trainable = False

# Define a batch processing
def process_images_in_batches(images, labels, batch_size=32):
    resize_layer = Resizing(224, 224)
    features_list = []
    total_batches = len(images) // batch_size + (1 if len(images) % batch_size > 0 else 0)
    
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size].astype('float32')
        
        # Resize
        batch_resized = resize_layer(batch_images)
        
        # Preprocess for VGG16
        batch_preprocessed = preprocess_input(batch_resized)
        
        # Extract features
        batch_features = base_model.predict(batch_preprocessed, verbose=1)
        
        # Flatten features
        batch_features_flat = batch_features.reshape(batch_features.shape[0], -1)
        features_list.append(batch_features_flat)
        
        print(f"Processed batch {i//batch_size + 1}/{total_batches}")
    
    # Concatenate all batches
    features = np.concatenate(features_list)
    return features, labels


# Process training, validation and test data
print("Processing training data...")
train_features, train_labels = process_images_in_batches(x_train, y_train)
print("Processing validation data...")
val_features, val_labels = process_images_in_batches(x_val, y_val)
print("Processing test data...")
test_features, test_labels = process_images_in_batches(x_test, y_test)

# Print shapes to confirm processing worked
print(f"Train features shape: {train_features.shape}, labels shape: {train_labels.shape}")
print(f"Validation features shape: {val_features.shape}, labels shape: {val_labels.shape}")
print(f"Test features shape: {test_features.shape}, labels shape: {test_labels.shape}")

Processing training data...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 1/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 2/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 3/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 4/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 5/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 6/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 7/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 8/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 9/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 10/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 11/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 12/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 13/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 14/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 15/1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Processed batch 16/1250
1/1 ━━━━━━━━━━━━━━━━━